In [17]:
import csv
import math
import numpy as np
import pandas as pd 
import itertools as itr 


def apriori(minsup, minconf, minlift, file):

    def readFile(filename):
        with open(filename, 'r', encoding = 'utf-8-sig') as theFile:
            row = csv.reader(theFile, delimiter=',')
            data = list(itr.islice(row, None))
        return data

#---------------------------------------------------------

    def getFrequency(data, transactions): 

        itemFrequency = {}
        iterData = itr.cycle(data)

        if type(data) == np.ndarray:
            counter = 0
            while counter < len(data):
                trans = list(next(iterData))
                iterRow = itr.cycle(trans)

                counter2 = 0
                while counter2 < len(trans):
                    
                    item = next(iterRow)
                    theItem = ""

                    if type(item) == str:
                        theItem = (item)
                    else:
                        theItem = tuple(item)

                    if theItem in itemFrequency:
                        itemFrequency[theItem] += 1
                    else:
                        itemFrequency[theItem] = 1

                    counter2 += 1
                
                counter += 1
        
            itemFrequency["RowCount"] = counter

        else:
            setOfTuples = set(data)
            iterData = itr.cycle(setOfTuples)
            
            counter = 0
            while counter < len(data):
                itemGroup = set(next(iterData))
                itemGroupChanged = tuple(itemGroup)

                itemFrequency[itemGroupChanged] = 0.1
                counter += 1

            cycleTransactions = itr.cycle(transactions)

            counter2 = 0
            while counter2 < len(transactions):
                sale = next(cycleTransactions)
                for item in itemFrequency:

                    setSale = set(sale)
                    setItemGroup = set(item)

                    if setItemGroup.issubset(setSale):
                        itemFrequency[item] += 1 

                counter2 += 1

            delete = []
            for item, count in itemFrequency.items():
                if count == 0.1:
                    delete.append(item)
                else:
                    baseVal = itemFrequency[item]
                    cleanVal = baseVal - 0.1
                    fixedVal = round(cleanVal)
                    toDict = int(fixedVal)
                    itemFrequency[item] = toDict

            for delItem in delete:
                itemFrequency.pop(delItem)

        return itemFrequency

#----------------------------------------------------------

    def mostFrequent(frequencyDict, minsup, rowCount):
        delete = []

        for item, count in frequencyDict.items():
            if count/rowCount < minsup:
                delete.append(item)

        for delItem in delete:
            frequencyDict.pop(delItem)
        
        return frequencyDict

#----------------------------------------------------------

    def getSupport(frequencyDict, rowCount):
        supDict = {}

        for item, count in frequencyDict.items():
            support = count/rowCount

            if item in supDict:
                supDict[item] += support
            else:
                supDict[item] = support

        return supDict

#----------------------------------------------------------

    def associationRules(prunedNDicts, supportDicts):

        frequentItemsetList = []
        rulesDict = {}

    #------------------------------------------------------

        def pruneConfidence(minConf, LHS, RHS, supportDicts):
            LHS = set(LHS)
            RHS = set(RHS)
            both = LHS.union(RHS)

            if len(LHS) == 1:
                LHS = ''.join(LHS)
            else:
                LHS = tuple(LHS)

            both = tuple(both)

            LHSSup = 0
            bothSup = 0
            for i in range(len(supportDicts)):
                for ante, conse in supportDicts[i].items():
                    if set(LHS) == set(ante):
                        LHSSup += supportDicts[i][ante]

                    if set(both) == set(ante):
                        bothSup += supportDicts[i][ante]

            conf = bothSup / LHSSup

            return conf

    #------------------------------------------------------

        def pruneLift(minLift, LHS, RHS, supportDicts):
            LHS = set(LHS)
            RHS = set(RHS)
            both = LHS.union(RHS)

            if len(LHS) == 1:
                LHS = ''.join(LHS)
            else:
                LHS = tuple(LHS)

            if len(RHS) == 1:
                RHS = ''.join(RHS)
            else:
                RHS = tuple(RHS)

            both = tuple(both)

            LHSSup = 0
            RHSSup = 0
            bothSup = 0
            for i in range(len(supportDicts)):
                for ante, conse in supportDicts[i].items():
                    if set(LHS) == set(ante):
                        LHSSup += supportDicts[i][ante]

                    if set(RHS) == set(ante):
                        RHSSup += supportDicts[i][ante]

                    if set(both) == set(ante):
                        bothSup += supportDicts[i][ante]

            lift = bothSup / (LHSSup * RHSSup)

            return lift
  
    #------------------------------------------------------    
    
        def getSubsets(bigSet, someNum):
            subsetList = []
            subsetLength = someNum

            if someNum > 0:
                for subset in itr.combinations(bigSet, subsetLength):
                    subsetList.append(subset)

                subsetList += getSubsets(bigSet, subsetLength-1)

            return subsetList

        flippedDicts = prunedNDicts[::-1]

        for i in range(len(flippedDicts)):
            for itemset in flippedDicts[i].keys():
                if i == len(flippedDicts)-1:
                    break

                else:
                    frequentItemset = list(itemset)
                    frequentItemsetList += [getSubsets(frequentItemset, len(frequentItemset))]

        ruleCount = 0
        for transactionSets in frequentItemsetList:
            toSet = []

            for item in transactionSets:
                toSet.append(set(item))

            superSet = toSet[0]
            
            for smallSet in toSet:

                LHS = tuple(smallSet)
                RHS = tuple(superSet - smallSet)

                if superSet - smallSet == set():
                    continue

                ruleConf = pruneConfidence(minconf, LHS, RHS, supportDicts)
                if ruleConf < minconf:
                    continue 
                
                ruleLift = pruneLift(minlift, LHS, RHS, supportDicts)
                if ruleLift < minlift:
                    continue

                else:
                    ruleLen = len(smallSet) + len(superSet - smallSet)
                    rule = tuple([tuple(smallSet), tuple(superSet - smallSet)])
                    rulesDict[rule] = [ruleLen, ruleLift, ruleConf]
                    ruleCount += 1
        

        sortRules = sorted((value, key) for (key, value) in rulesDict.items())
        sortedRules = dict([(k,v) for v, k in sortRules])

        for finalRule, finalVals in reversed(sortedRules.items()):
            print(set(finalRule[0]), ' ==> ', set(finalRule[1])
                ,'\n'
                ,'Rule Size: '
                ,finalVals[0]
                ,'\n'
                ,'Rule Lift: '
                ,finalVals[1]
                ,'\n'
                ,'Rule Confidence: '
                ,finalVals[2]
                ,'\n')
        
        print('Total Rules: ', ruleCount)

#----------------------------------------------------------

    allFreqDicts = []
    allSupDicts = []

    theData = np.asarray(readFile(file), dtype = object)

    frequencyDict = getFrequency(theData, None)

    totalRows = frequencyDict["RowCount"]
    frequencyDict.pop("RowCount")

    totalItems = len(frequencyDict.keys())

    singleSupport = getSupport(frequencyDict, totalRows)
    allSupDicts.append(singleSupport)

    singleMostFrequent = mostFrequent(frequencyDict, minsup, totalRows)
    allFreqDicts.append(singleMostFrequent)

    itemsetLength = len(singleMostFrequent)
    
    currentFreqDict = singleMostFrequent
    for i in range(2, totalItems):

        remainingItems = []

        for item in currentFreqDict:
            if type(item) == tuple:
                for value in item:
                    remainingItems.append(item)

            else:
                remainingItems.append(item)

        if len(remainingItems) == 0:
            break

        else:
            remainingItemsNP = np.array(remainingItems)

            uniqueListRemaining = list(np.unique(remainingItemsNP))

            itemCombos = list(itr.combinations(uniqueListRemaining, i))

            currentTotalFreq = getFrequency(itemCombos, theData)
            cutTotalFreq = mostFrequent(currentTotalFreq, minsup, totalRows)
            print(cutTotalFreq)

            itemsetLength += len(cutTotalFreq)

            currentSup = getSupport(cutTotalFreq, totalRows)

            if len(cutTotalFreq) != 0:
                allFreqDicts.append(cutTotalFreq)

            if len(currentSup) != 0:
                allSupDicts.append(currentSup)

            currentFreqDict = cutTotalFreq

    getTheRules = associationRules(allFreqDicts, allSupDicts)

    print('Total Frequent Subsets: ', itemsetLength)


#----------------------------------------------------------

apriori(0.15,0.8,1,'TASK1.csv')    



{('E', 'B'): 1, ('F', 'C'): 1, ('D', 'C'): 3, ('F', 'A'): 1, ('A', 'D'): 3, ('F', 'B'): 1, ('B', 'D'): 3, ('A', 'C'): 3, ('E', 'C'): 1, ('F', 'D'): 1, ('B', 'A'): 4, ('B', 'C'): 4}
{('E', 'B', 'C'): 1, ('B', 'F', 'A'): 1, ('F', 'B', 'C'): 1, ('F', 'B', 'D'): 1, ('B', 'A', 'D'): 3, ('B', 'D', 'C'): 3, ('F', 'A', 'C'): 1, ('F', 'A', 'D'): 1, ('F', 'D', 'C'): 1, ('A', 'D', 'C'): 3, ('B', 'A', 'C'): 3}
{('B', 'A', 'D', 'C'): 3, ('F', 'A', 'D', 'C'): 1, ('F', 'B', 'D', 'C'): 1, ('B', 'F', 'A', 'C'): 1, ('B', 'F', 'A', 'D'): 1}
{('B', 'F', 'D', 'A', 'C'): 1}
{'F', 'D'}  ==>  {'B', 'A', 'C'} 
 Rule Size:  5 
 Rule Lift:  1.6666666666666667 
 Rule Confidence:  1.0 

{'F', 'C'}  ==>  {'B', 'D', 'A'} 
 Rule Size:  5 
 Rule Lift:  1.6666666666666667 
 Rule Confidence:  1.0 

{'B', 'F', 'D'}  ==>  {'A', 'C'} 
 Rule Size:  5 
 Rule Lift:  1.6666666666666667 
 Rule Confidence:  1.0 

{'B', 'F', 'C'}  ==>  {'A', 'D'} 
 Rule Size:  5 
 Rule Lift:  1.6666666666666667 
 Rule Confidence:  1.0 

{'B', 'F'